In [173]:
import csv
import copy

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [174]:
dataset = './point_history.csv'
model_save_path = './gesture_classifier.hdf5'

In [175]:
TIME_STEPS = 16
DIMENSION = 2
NUM_CLASSES = 3

In [176]:
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (TIME_STEPS * DIMENSION) + 1 + 2)))

In [177]:
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))

In [178]:
for dataset_index, X_data in enumerate(X_dataset):
    for data_index in range(0, len(X_data), 2):
        if data_index == 0:
            image_width = X_data[data_index]
            image_height = X_data[data_index + 1]
            continue

        if data_index == 2:
            base_x = X_data[data_index]
            base_y = X_data[data_index + 1]

        X_data[data_index] = (X_data[data_index] - base_x) / image_width
        X_data[data_index + 1] = (X_data[data_index + 1] - base_y) / image_height

In [179]:
X_dataset = np.delete(X_dataset, [0, 1, 2, 3], 1)

In [180]:
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=42)

In [181]:
input_size = (TIME_STEPS * DIMENSION) - 2

model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(input_size, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [182]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout_12 (Dropout)        (None, 30)                0         
                                                                 
 dense_18 (Dense)            (None, 24)                744       
                                                                 
 dropout_13 (Dropout)        (None, 24)                0         
                                                                 
 dense_19 (Dense)            (None, 10)                250       
                                                                 
 dense_20 (Dense)            (None, 3)                 33        
                                                                 
Total params: 1,027
Trainable params: 1,027
Non-trainable params: 0
_________________________________________________________________


In [183]:
# モデルコンパイル
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [184]:
# モデルチェックポイントのコールバック
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# 早期打ち切り用コールバック
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [185]:
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
1/1 [==============================] - ETA: 0s - loss: 1.0694 - accuracy: 0.6250
Epoch 00001: saving model to ./gesture_classifier.hdf5
1/1 [==============================] - 0s 469ms/step - loss: 1.0694 - accuracy: 0.6250 - val_loss: 1.0749 - val_accuracy: 1.0000
Epoch 2/1000
1/1 [==============================] - ETA: 0s - loss: 1.0596 - accuracy: 0.7188
Epoch 00002: saving model to ./gesture_classifier.hdf5
1/1 [==============================] - 0s 33ms/step - loss: 1.0596 - accuracy: 0.7188 - val_loss: 1.0661 - val_accuracy: 1.0000
Epoch 3/1000
1/1 [==============================] - ETA: 0s - loss: 1.0593 - accuracy: 0.7812
Epoch 00003: saving model to ./gesture_classifier.hdf5
1/1 [==============================] - 0s 31ms/step - loss: 1.0593 - accuracy: 0.7812 - val_loss: 1.0567 - val_accuracy: 1.0000
Epoch 4/1000
1/1 [==============================] - ETA: 0s - loss: 1.0439 - accuracy: 0.8438
Epoch 00004: saving model to ./gesture_classifier.hdf5
1/1 [==============

In [186]:
# 推論テスト
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))

[9.9997067e-01 2.3629471e-05 5.6908693e-06]
0


In [187]:
%%timeit

# 時間計測
predict_result = model.predict(np.array([X_test[0]]))

33.7 ms ± 2.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [188]:
# 推論専用のモデルとして保存
model.save(model_save_path, include_optimizer=False)

In [189]:
model = tf.keras.models.load_model(model_save_path)

In [190]:
tflite_save_path = './gesture_classifier.tflite'

In [191]:
# モデルを変換(量子化)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open(tflite_save_path, 'wb').write(tflite_quantized_model)

INFO:tensorflow:Assets written to: /var/folders/_c/yfkk358d56n84q0rcrprvkc00000gn/T/tmpyw1edoe7/assets


INFO:tensorflow:Assets written to: /var/folders/_c/yfkk358d56n84q0rcrprvkc00000gn/T/tmpyw1edoe7/assets
2022-01-18 16:36:18.630997: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:363] Ignored output_format.
2022-01-18 16:36:18.631014: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:366] Ignored drop_control_dependency.
2022-01-18 16:36:18.631130: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /var/folders/_c/yfkk358d56n84q0rcrprvkc00000gn/T/tmpyw1edoe7
2022-01-18 16:36:18.632416: I tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2022-01-18 16:36:18.632428: I tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: /var/folders/_c/yfkk358d56n84q0rcrprvkc00000gn/T/tmpyw1edoe7
2022-01-18 16:36:18.637552: I tensorflow/cc/saved_model/loader.cc:210] Restoring SavedModel bundle.
2022-01-18 16:36:18.655232: I tensorflow/cc/saved_model/loader.cc:194] Running initializatio

6112

In [192]:
interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()

In [193]:
# 入出力テンソルを取得
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [194]:
# 推論実施
interpreter.set_tensor(input_details[0]['index'], np.array([X_dataset[0]]))
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

print(np.squeeze(tflite_results))
print(np.argmax(np.squeeze(tflite_results)))

[9.9986351e-01 1.0810874e-04 2.8352109e-05]
0


In [195]:
%%timeit
# 推論実施
interpreter.set_tensor(input_details[0]['index'], np.array([X_dataset[0]]))
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

6.32 µs ± 389 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
